In [1]:
import spacy
import os
import codecs
from collections import Counter, defaultdict
import pickle

filename = '/mnt/drive1/data/eco/NAIL_DATAFIELD_txt/parsed_v3/parsed_v3_valid.txt'
# starting from the expoeriments folder
filename = '../../data/NAIL_DATAFIELD_txt/parsed_v3/arts_arthistory_aesthetics_valid.txt'
if not os.path.isfile(filename):
    print "file",filename,'is not there...'

In [21]:
# helper functions
def read_file(file_name):
    with open(file_name, 'r') as file:
        return file.read().decode('utf-8')
    
def stream_file(file_name):
    with codecs.open(file_name, 'r', 'utf-8') as file:
        for line in file:
            yield line

def write_lists_to_file(lists):
    with codecs.open('character_file.txt','w','utf-8') as out:
        for al in lists:
            try:
                out.write(al[0]+','+str(al[1])+'\n')
            except:
                print 'error with',al
    # safety dump
    with open('character.dump','w') as dump_file:
        pickle.dump(characters,dump_file)
    
def find_character_occurences(processed_txt):
    """
    Return a list of actors from `doc` with corresponding occurences.
    """
    total_len = len(processed_txt)
    characters = Counter()
    
    index = 0
    for ent in processed_txt.ents:
        if ent.label_ == 'PERSON':
            characters[ent.lemma_] += 1
        if index % (total_len/10) == 0:
            print '*',
        index += 1
            
    return characters

In [22]:
# tests
file = stream_file(filename)
file.next()

u'The first three volumes provide a broad historic and philosophical framework for the understanding of the arts in education .\n'

In [23]:
# higher helper functions

class spacy_helper:
    
    def __init__(self, lang='en'):
        
        self.nlp = spacy.load(lang) 
        self.in_generator = None
        
    def spacy_process_file(self, file_name):
        self.in_generator = stream_file(file_name)
        for line in self.in_generator:
            yield self.nlp(line)
    

In [24]:
sup_spacy = spacy_helper()


The first three volumes provide a broad historic and philosophical framework for the understanding of the arts in education .

In [30]:
space_gen = sup_spacy.spacy_process_file(filename)
person = None
while not person:
    sentence = space_gen.next()
    print sentence

    for ent in sentence.ents:
        print ent, ent.label_
        if ent.label_ == 'PERSON':
            print ent, '$$$'
            person = ent
            break


The first three volumes provide a broad historic and philosophical framework for the understanding of the arts in education .

first ORDINAL
three CARDINAL
The subsequent volumes elaborate the implications of this comprehensive aesthetic for each of the six major art disciplines and for the teaching of the arts in the primary school .

six CARDINAL
Library of Congress Cataloging-in-Publication Data The Symbolic order : a contemporary reader on the arts debate edited by Peter Abbs. p. cm .

Peter Abbs PERSON
Peter Abbs $$$


In [91]:
# open the file
a = spacy_helper()
# text = read_file(filename)
# print 'text read'
# # Process `text` with Spacy NLP Parser
# processed_text = nlp(text)
# print 'text processed'

In [92]:

for i in range(1):
    sentence = space_gen.next()
    a.nlp.parser(sentence)
#     parsed = 
    print sentence
    print type(sentence)
    print '#########'
    for t in sentence:
        print t,t.pos_, t.tag_, t.dep_, t.sentiment
#         print 'ent_type_',t.ent_type_
#         print 'lemma_',t.lemma_
#         print 'shape', t.shape_
#         ent = token.ent_
#         print type(ent)
#         print '--'
#         print ent, ent.label_
#         print 'lemma', ent.lemma_
#         print 'root.dep', ent.root.dep_
#         for child in ent.root.head.children:
#             print '-',child


He ’ d approve the inverted commas , believing that you can ’ t teach poetry , only ‘ the craft of verse… ’ .

<type 'spacy.tokens.doc.Doc'>
#########
He PRON PRP nsubj 0.0
’ PUNCT . punct 0.0
d X XX aux 0.0
approve VERB VB ROOT 0.0
the DET DT det 0.0
inverted ADJ JJ amod 0.0
commas NOUN NN dobj 0.0
, PUNCT , punct 0.0
believing VERB VBG advcl 0.0
that ADP IN mark 0.0
you PRON PRP nsubj 0.0
can VERB MD aux 0.0
’ VERB VB ccomp 0.0
t PRON PRP dobj 0.0
teach VERB VB conj 0.0
poetry NOUN NN dobj 0.0
, PUNCT , punct 0.0
only ADV RB advmod 0.0
‘ VERB MD ccomp 0.0
the DET DT det 0.0
craft NOUN NN dobj 0.0
of ADP IN prep 0.0
verse NOUN NN pobj 0.0
… PUNCT NFP punct 0.0
’ PUNCT NFP punct 0.0
. PUNCT . punct 0.0

 SPACE SP  0.0


In [ ]:
characters = find_character_occurences(processed_text)
print 'characters found'
write_lists_to_file(characters)
print 'characters written. done for today'

In [ ]:
doc = nlp(u'Hello, Rubert. Natural Language Processing in 10 lines of code.')
characters = find_character_occurences(doc)
print characters
write_lists_to_file(characters)


In [ ]:
# lets have some recursive folder parsing
# in the doc2vec_helper
base_dir = '../../data/parsed_v3/'
base_dir += 'arts_arthistory_aesthetics'

def path(base_path,file_name):
    if not base_path.endswith('/'):
        base_path += '/'
    return base_path + file_name

def get_dirs(base):
    return [f for f in os.listdir(base) if os.path.isdir(path(base,f))]

def get_files(base):
    if not base.endswith('/'):
        base += '/'
    return [f for f in os.listdir(base) if os.path.isfile(path(base,f))]    

def my_filter(base_path,file_list):
    def cool_file(path):
        return path.endswith('.txt') and os.stat(path).st_size > 0        
    return [f for f in file_list if cool_file(path(base_path,f))]  

folder_list = get_dirs(base_dir)
print folder_list
files =  my_filter(base_dir,get_files(base_dir))

In [ ]:
files[0]

In [ ]:
### feeding depency parse into a rnn in order to detect broken sentences
text = read_file(folder_list)
processed_text = nlp(text)